In [19]:
import numpy as np
from statistics import mean
import spacy
from datetime import date
from spacytextblob.spacytextblob import SpacyTextBlob
import warnings
import pandas as pd
import requests
import json
from datetime import datetime, timedelta
from langchain.chat_models import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate
from langchain.schema.messages import SystemMessage
from langchain.prompts import ChatPromptTemplate
from secret_key import api_key
warnings.filterwarnings('ignore')

In [2]:
tick = 'AAPL'

def get_stock_news(date, stock, api_key) -> pd.DataFrame:
    date_7_ago = date - timedelta(days=7)
    date_7_ago = date_7_ago.strftime('%Y-%m-%d')
    url = f'https://eodhistoricaldata.com/api/news?api_token={api_key}&s={stock}&from={date_7_ago}&to={date}&limit=1000'
    news_json = requests.get(url).json()
    news_data = pd.DataFrame(news_json)
    news_data = news_data.drop(['sentiment'], axis = 1)
    news_data['date'] = news_data['date'].apply(lambda x: datetime.fromisoformat(x).strftime(r'%Y-%m-%d'))
    d = {date:[] for date in news_data['date'].unique()}
    for index,row in news_data.iterrows():
        row_date = row['date']
        d[row_date].append(row['title'])
    date_col = [key for key, value in d.items()]
    news_col = [value for key, value in d.items()]
    return pd.DataFrame({'date':date_col,'news_col':news_col})

date = date.today()
api_key = '654a55727061d0.65500407'

news_data = get_stock_news(date, tick, api_key)

In [3]:
news_data

,date,news_col
0,2023-11-08,"[1 Semiconductor Stock Set to Join Apple, Amaz..."
1,2023-11-07,[Assessing the Ownership Landscape of Apple In...
2,2023-11-06,"[15 Best Cheap Wireless Earbuds Under $50, Dow..."
3,2023-11-05,[Big Tech’s Growth Status in Doubt With Weaker...
4,2023-11-04,"[Starbucks leans into Apple, Microsoft, Amazon..."
5,2023-11-03,[Apple TV+ Is a Bright Spot in Apple's Mixed Q...
6,2023-11-02,"[Asian stocks rally tracking Wall St strength,..."
7,2023-11-01,[Dow Jones Futures Rise: Market Rally Gains St...


In [10]:
def adding_sent_subj(df, ticker):
    nlp = spacy.load('en_core_web_sm')
    nlp.add_pipe('spacytextblob')

    daily_sentiment_col = []
    daily_subjectivity_col = []
    for index, row in df.iterrows():
        daily_sentiment_list = []
        daily_subjectivity_list = []
        for news in row['news_col']:
            doc = nlp(news)
            daily_sentiment_list.append(doc._.blob.polarity)
            daily_subjectivity_list.append(doc._.blob.subjectivity)
        daily_sentiment_col.append(np.mean(daily_sentiment_list))
        daily_subjectivity_col.append(np.mean(daily_subjectivity_list))
    df['daily_sentiment'] = daily_sentiment_col
    df['daily_subjectivity'] = daily_subjectivity_col

    df.to_csv(f'data/{ticker}.csv')
    print('Saving dataset to "data" folder!\n\n')

    return df[::-1]


    

news_data_sent_subj = adding_sent_subj(news_data, tick)
news_data_sent_subj

Saving dataset to "data" folder!




,date,news_col,daily_sentiment,daily_subjectivity
7,2023-11-01,[Dow Jones Futures Rise: Market Rally Gains St...,0.166667,0.358108
6,2023-11-02,"[Asian stocks rally tracking Wall St strength,...",0.040770,0.144821
5,2023-11-03,[Apple TV+ Is a Bright Spot in Apple's Mixed Q...,0.050225,0.254516
4,2023-11-04,"[Starbucks leans into Apple, Microsoft, Amazon...",0.224405,0.309127
3,2023-11-05,[Big Tech’s Growth Status in Doubt With Weaker...,0.165774,0.220833
2,2023-11-06,"[15 Best Cheap Wireless Earbuds Under $50, Dow...",0.060268,0.218750
1,2023-11-07,[Assessing the Ownership Landscape of Apple In...,0.122538,0.225325
0,2023-11-08,"[1 Semiconductor Stock Set to Join Apple, Amaz...",0.144156,0.316883


In [22]:
llm = ChatOpenAI(openai_api_key=api_key,
                            model_name='gpt-3.5-turbo-16k',    # gpt-3.5-turbo | gpt-3.5-turbo-16k  | gpt-4
                            temperature=0.3,
                            request_timeout=120)

In [13]:
from secret_key import api_key  #DIVIDE BY TWO 
def subj_sent_analyzer(df):    
    template = ChatPromptTemplate.from_messages(
    [#SystemMessage(content=("Тебе нужно дать ответ ввиде python списка. Пример: ['item1', 'item2', 'item3'...]")),
     HumanMessagePromptTemplate.from_template(
        "Imagine you're a financial assistant. You've been given news for recent 2 days. Also you've been given sentiment and subjectivity rating of these news. Your task is to analyze future stock trend by these data.\n"
        "Sentiment for day 1: {SENTIMENT_1}\n"
        "Subjectivity for day 1: {SUBJECTIVITY_1}\n"
        "Sentiment for day 2: {SENTIMENT_2}\n"
        "Subjectivity for day 2: {SUBJECTIVITY_2}\n")])
    
    response = llm(template.format_messages(
                             SENTIMENT_1=df['daily_sentiment'].iloc[0], 
                             SUBJECTIVITY_1=df['daily_subjectivity'].iloc[0],
                             SENTIMENT_2=df['daily_sentiment'].iloc[1],
                             SUBJECTIVITY_2=df['daily_subjectivity'].iloc[1])).content
    return response

subj_sent_response = subj_sent_analyzer(news_data_sent_subj)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Wed, 08 Nov 2023 13:52:25 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '822e4b8368f51654-DME', 'alt-svc': 'h3=":443"; ma=86400'}.


In [21]:
def news_analyzer(df, api_key):
    template = ChatPromptTemplate.from_messages(
    [#SystemMessage(content=("Тебе нужно дать ответ ввиде python списка. Пример: ['item1', 'item2', 'item3'...]")),
     HumanMessagePromptTemplate.from_template(
        "Imagine you're a financial assistant. You've been given lists of news for recent 2 days. Your task is to analyze future stock trend by these data.\n"
        "List of news of the first day: {NEWS_1}\n"
        "List of news of the second day: {NEWS_2}\n")])
    
    response = llm(template.format_messages(
                            NEWS_1=df['news_col'].iloc[0], 
                            NEWS_2=df['news_col'].iloc[1])).content
    return response

news_response = news_analyzer(news_data_sent_subj)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Wed, 08 Nov 2023 13:58:08 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '822e54230ae49d37-DME', 'alt-svc': 'h3=":443"; ma=86400'}.
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad

APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Wed, 08 Nov 2023 13:59:13 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '822e556029339d37-DME', 'alt-svc': 'h3=":443"; ma=86400'}

In [ ]:
print(subj_sent_response)

Based on the given data, the sentiment and subjectivity ratings for the news over the past two days are as follows:

Day 1:
- Sentiment: 0.16666666666666669
- Subjectivity: 0.3581081081081081

Day 2:
- Sentiment: 0.040769675925925924
- Subjectivity: 0.14482060185185183

To analyze the future stock trend based on this data, we can consider the sentiment and subjectivity ratings as indicators. 

The sentiment rating represents the overall sentiment or positivity/negativity of the news. A positive sentiment rating indicates a positive outlook, while a negative sentiment rating suggests a negative outlook.

The subjectivity rating measures the subjectiveness or objectiveness of the news. A higher subjectivity rating indicates that the news is more opinion-based, while a lower subjectivity rating suggests that the news is more factual and objective.

Based on the given data, the sentiment ratings for both days are relatively low, indicating a neutral or slightly negative sentiment. This sug

In [ ]:
print(news_response)

NameError: name 'news_response' is not defined